In [ ]:
import numpy as np
from scipy import ndimage


In [ ]:
# org_images=np.load('train_images_task3_299.npy')
# org_gt=np.load('gt_labels_Task3_224.npy')
# org_images_shape = org_images.shape
# org_gt_shape = org_gt.shape

In [ ]:

# NOTE
# Images are assumed to be uint8 0-255 valued.
# For augment function:
#   images shape: (batch_size, height, width, channels=3)
#   labels shape: (batch_size, 3)
def addBlotch(image, max_dims=[0.2,0.2]):
    #add's small black/white box randomly in periphery of image
    new_image = np.copy(image)
    shape = new_image.shape
    max_x = shape[0] * max_dims[0]
    max_y = shape[1] * max_dims[1]
    rand_x = 0
    rand_y = np.random.randint(low=0, high=shape[1])
    rand_bool = np.random.randint(0,2)
    if rand_bool == 0:
        rand_x = np.random.randint(low=0, high=max_x)
    else:
        rand_x = np.random.randint(low=(shape[0]-max_x), high=shape[0])
    size = np.random.randint(low=1, high=7) #size of each side of box
    new_image[rand_x:(size+rand_x), rand_y:(size+rand_y), :] = np.random.randint(0,256)
    return new_image

def shift(image, max_amt=0.2):
    new_img = np.copy(image)
    shape = new_img.shape
    max_x = int(shape[0] * max_amt)
    max_y = int(shape[1] * max_amt)
    x = np.random.randint(low=-max_x, high=max_x)
    y = np.random.randint(low=-max_y, high=max_y)
    return ndimage.interpolation.shift(new_img,shift=[x,y,0])

def addNoise(image, amt=0.005):
    noise_mask = np.random.poisson(image / 255.0 * amt) / amt * 255
    noisy_img = image + (noise_mask)
    return np.array(np.clip(noisy_img, a_min=0., a_max=255.), dtype=np.uint8)

def rotate(image):
    randnum = np.random.randint(1,360)
    new_image = np.copy(image)
    return ndimage.rotate(new_image, angle=randnum, reshape=False)

#randomly manipulates image
#rotate, flip along axis, add blotch, shift
def augment(images, labels=None, amplify=2):
    # INPUT:
    #images shape: (batch_size, height, width, channels=3)
    #labels shape: (batch_size, 3)
    ops = {
        0: addBlotch,
        1: shift,
        2: addNoise,
        3: rotate
    }

    shape = images.shape
    new_images = np.zeros(((amplify*shape[0]), shape[1], shape[2], shape[3]))
    if labels is not None:
        new_labels = np.zeros(((amplify*shape[0]), 7))
    for i in range(images.shape[0]):
        cur_img = np.copy(images[i])
        new_images[i] = cur_img
        if labels is not None:
            new_labels[i] = np.copy(labels[i])
        for j in range(1, amplify):
            add_r = ( j * shape[0] )
            which_op = np.random.randint(low=0, high=4)
            dup_img = np.zeros((1,shape[1], shape[2], shape[3]))
            new_images[i+add_r] = ops[which_op](cur_img)
            if labels is not None:
                new_labels[i+add_r] = np.copy(labels[i])
    if labels is not None:
        return new_images.astype(np.uint8), new_labels.astype(np.uint8)
    else:
        return new_images.astype(np.uint8)

In [ ]:

# # NOTE
# # Images are assumed to be uint8 0-255 valued.
# # For augment function:
# #   images shape: (batch_size, height, width, channels=3)
# #   labels shape: (batch_size, 3)

# def addBlotch(image, max_dims=[0.2,0.2]):
#     #add's small black/white box randomly in periphery of image
#     new_image = np.copy(image)
#     shape = new_image.shape
#     max_x = shape[0] * max_dims[0]
#     max_y = shape[1] * max_dims[1]
#     rand_x = 0
#     rand_y = np.random.randint(low=0, high=shape[1])
#     rand_bool = np.random.randint(0,2)
#     if rand_bool == 0:
#         rand_x = np.random.randint(low=0, high=max_x)
#     else:
#         rand_x = np.random.randint(low=(shape[0]-max_x), high=shape[0])
#     size = np.random.randint(low=1, high=7) #size of each side of box
#     new_image[rand_x:(size+rand_x), rand_y:(size+rand_y), :] = np.random.randint(0,256)
#     return new_image

# def shift(image, max_amt=0.2):
#     new_img = np.copy(image)
#     shape = new_img.shape
#     max_x = int(shape[0] * max_amt)
#     max_y = int(shape[1] * max_amt)
#     x = np.random.randint(low=-max_x, high=max_x)
#     y = np.random.randint(low=-max_y, high=max_y)
#     return ndimage.interpolation.shift(new_img,shift=[x,y,0])

# def addNoise(image, amt=0.005):
#     noise_mask = np.random.poisson(image / 255.0 * amt) / amt * 255
#     noisy_img = image + (noise_mask)
#     return np.array(np.clip(noisy_img, a_min=0., a_max=255.), dtype=np.uint8)

# def rotate(image):
#     randnum = np.random.randint(1,360)
#     new_image = np.copy(image)
#     return ndimage.rotate(new_image, angle=randnum, reshape=False)

# #randomly manipulates image
# #rotate, flip along axis, add blotch, shift
# def augment(images, labels=None, amplify=2):
#     # INPUT:
#     #images shape: (batch_size, height, width, channels=3)
#     #labels shape: (batch_size, 3)
#     ops = {
#         0: addBlotch,
#         1: shift,
#         2: addNoise,
#         3: rotate
#     }
#     shape = images.shape
#     new_images = np.zeros(((org_images_shape[0]+(amplify-1)*shape[0]), shape[1], shape[2], shape[3]))
#     if labels is not None:
#         new_labels = np.zeros(((org_images_shape[0]+(amplify-1)*shape[0]), 7))
#     for i in range(images.shape[0]):
#         cur_img = np.copy(images[i])
#         new_images[org_images_shape[0]+i] = cur_img
#         if labels is not None:
#             new_labels[org_gt_shape[0]+i] = np.copy(labels[i])
#         for j in range(1, amplify-1):
#             add_r = ( j * shape[0] )
#             which_op = np.random.randint(low=0, high=4)
#             dup_img = np.zeros((1,shape[1], shape[2], shape[3]))
#             new_images[org_images_shape[0]+i+add_r] = ops[which_op](cur_img)   # augmentation part
#             if labels is not None:
#                 new_labels[org_images_shape[0]+i+add_r] = np.copy(labels[i])
#     if labels is not None:
#         return new_images.astype(np.uint8), new_labels.astype(np.uint8)
#     else:
#         return new_images.astype(np.uint8)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
import numpy as np
import scipy.io as scio    
import os,re
from PIL import Image
from scipy import ndimage
from matplotlib import pyplot as plt
import cv2

In [ ]:
IMAGE_SIZE = 299

In [ ]:
def atoi(text) : 
    return int(text) if text.isdigit() else text

In [ ]:
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]

In [ ]:
def get_filenames(path):
    filenames = []
    for root, dirnames, filenames in os.walk(path):
        filenames.sort(key = natural_keys)
        rootpath = root
    print(len(filenames))
    return filenames

In [ ]:
root_path = "/Users/syh4661/workspace/Preprocessing/"

In [ ]:
images_MEL=np.load('train_images_task3_299_MEL.npy')

In [ ]:
images_NV=np.load('train_images_task3_299_NV.npy')

In [ ]:
images_BCC=np.load('train_images_task3_299_BCC.npy')

In [ ]:
images_AKIEC=np.load('train_images_task3_299_AKIEC.npy')

In [ ]:
images_BKL=np.load('train_images_task3_299_BKL.npy')

In [ ]:
images_DF=np.load('train_images_task3_299_DF.npy')

In [ ]:
images_VASC=np.load('train_images_task3_299_VASC.npy')

In [ ]:
gt_MEL=np.load('gt_labels_Task3_%d_MEL.npy'%IMAGE_SIZE)
gt_NV=np.load('gt_labels_Task3_%d_NV.npy'%IMAGE_SIZE)
gt_BCC=np.load('gt_labels_Task3_%d_BCC.npy'%IMAGE_SIZE)
gt_AKIEC=np.load('gt_labels_Task3_%d_AKIEC.npy'%IMAGE_SIZE)
gt_BKL=np.load('gt_labels_Task3_%d_BKL.npy'%IMAGE_SIZE)
gt_DF=np.load('gt_labels_Task3_%d_DF.npy'%IMAGE_SIZE)
gt_VASC=np.load('gt_labels_Task3_%d_VASC.npy'%IMAGE_SIZE)


In [ ]:
a=images_MEL[:3]

In [ ]:
a.shape

In [ ]:
plt.imshow(a[0], cmap = "gray")
plt.show()

In [ ]:
plt.imshow(a[1], cmap = "gray")
plt.show()

In [ ]:
plt.imshow(a[2], cmap = "gray")
plt.show()

In [ ]:
b=augment(a,amplify=4)

In [ ]:
b.shape

In [ ]:
plt.imshow(b[6], cmap = "gray")
plt.show()

In [ ]:
gt_MEL.shape

In [ ]:
images_MEL.shape

In [ ]:
images_MEL_aug.shape

In [ ]:
images_MEL_aug, gt_MEL_aug = augment(images_MEL,gt_MEL,amplify=6)

In [ ]:
images_BCC_aug, gt_BCC_aug = augment(images_BCC,gt_BCC, amplify=13)
images_AKIEC_aug, gt_AKIEC_aug = augment(images_AKIEC,gt_AKIEC, amplify=20)
images_BKL_aug, gt_BKL_aug = augment(images_BKL,gt_BKL, amplify=6)
images_DF_aug, gt_DF_aug = augment(images_DF,gt_DF, amplify=58)
images_VASC_aug, gt_VASC_aug = augment(images_VASC,gt_VASC, amplify=47)

In [ ]:
np.save('train_images_task3_%d_MEL_aug.npy'%IMAGE_SIZE,images_MEL_aug)
# np.save('train_images_task3_%d_NV_aug.npy'%IMAGE_SIZE,train_images_NV)
np.save('train_images_task3_%d_BCC_aug.npy'%IMAGE_SIZE,images_BCC_aug)
np.save('train_images_task3_%d_AKIEC_aug.npy'%IMAGE_SIZE,images_AKIEC_aug)
np.save('train_images_task3_%d_BKL_aug.npy'%IMAGE_SIZE,images_BKL_aug)
np.save('train_images_task3_%d_DF_aug.npy'%IMAGE_SIZE,images_DF_aug)
np.save('train_images_task3_%d_VASC_aug.npy'%IMAGE_SIZE,images_VASC_aug)

In [ ]:
np.save('gt_labels_Task3_%d_MEL_aug.npy'%IMAGE_SIZE,gt_MEL)
# np.save('gt_labels_Task3_%d_NV_aug.npy'%IMAGE_SIZE,gt_NV)
np.save('gt_labels_Task3_%d_BCC_aug.npy'%IMAGE_SIZE,gt_BCC)
np.save('gt_labels_Task3_%d_AKIEC_aug.npy'%IMAGE_SIZE,gt_AKIEC)
np.save('gt_labels_Task3_%d_BKL_aug.npy'%IMAGE_SIZE,gt_BKL)
np.save('gt_labels_Task3_%d_DF_aug.npy'%IMAGE_SIZE,gt_DF)
np.save('gt_labels_Task3_%d_VASC_aug.npy'%IMAGE_SIZE,gt_VASC)

In [ ]:
images_VASC_aug.shape

In [ ]:
images_MEL_aug=np.load('train_images_task3_%d_MEL_aug.npy'%IMAGE_SIZE)
images_NV_aug=np.load('train_images_task3_%d_NV.npy'%IMAGE_SIZE)
images_BCC_aug=np.load('train_images_task3_%d_BCC_aug.npy'%IMAGE_SIZE)
images_AKIEC_aug=np.load('train_images_task3_%d_AKIEC_aug.npy'%IMAGE_SIZE)
images_BKL_aug=np.load('train_images_task3_%d_BKL_aug.npy'%IMAGE_SIZE)
images_DF_aug=np.load('train_images_task3_%d_DF_aug.npy'%IMAGE_SIZE)
images_VASC_aug=np.load('train_images_task3_%d_VASC_aug.npy'%IMAGE_SIZE)

In [ ]:
gt_MEL=np.load('gt_labels_Task3_%d_MEL_aug.npy'%IMAGE_SIZE)
gt_NV=np.load('gt_labels_Task3_%d_NV.npy'%IMAGE_SIZE)
gt_BCC=np.load('gt_labels_Task3_%d_BCC_aug.npy'%IMAGE_SIZE)
gt_AKIEC=np.load('gt_labels_Task3_%d_AKIEC_aug.npy'%IMAGE_SIZE)
gt_BKL=np.load('gt_labels_Task3_%d_BKL_aug.npy'%IMAGE_SIZE)
gt_DF=np.load('gt_labels_Task3_%d_DF_aug.npy'%IMAGE_SIZE)
gt_VASC=np.load('gt_labels_Task3_%d_VASC_aug.npy'%IMAGE_SIZE)

In [ ]:
print gt_MEL.shape
print gt_NV.shape
print gt_BCC.shape
print gt_AKIEC.shape
print gt_BKL.shape
print gt_DF.shape
print gt_VASC.shape

In [ ]:
x=np.array([])

In [ ]:
for i in range(6):
    x=np.append(x,gt_MEL)
    
del gt_MEL

In [ ]:
x=np.append(x,gt_NV)
del gt_NV

In [ ]:
for i in range(13):
    x=np.append(x,gt_BCC)
    
del gt_BCC

In [ ]:
for i in range(20):
    x=np.append(x,gt_AKIEC)
    
del gt_AKIEC


In [ ]:
for i in range(6):
    x=np.append(x,gt_BKL)
    
del gt_BKL

In [ ]:
for i in range(58):
    x=np.append(x,gt_DF)
del gt_DF

In [ ]:
for i in range(47):
    x=np.append(x,gt_VASC)
del gt_VASC

In [ ]:
x=x.reshape((46543,7))

In [ ]:
x.shape

In [ ]:
np.save('gt_task3_%d_aug.npy'%IMAGE_SIZE,x)

In [ ]:
tot=images_MEL_aug.shape[0]+images_NV_aug.shape[0]+images_BCC_aug.shape[0]+images_AKIEC_aug.shape[0]+images_BKL_aug.shape[0]+images_DF_aug.shape[0]+images_VASC_aug.shape[0]

In [ ]:
tot

In [ ]:
x=np.append(gt_MEL,gt_NV)

In [ ]:
x=np.append(x,gt_BCC)

In [ ]:
x=np.append(x,gt_AKIEC)

In [ ]:
x=np.append(x,gt_BKL)

In [ ]:
x=np.append(x,gt_DF)

In [ ]:
x=np.append(x,gt_VASC)

In [ ]:
x=x.reshape((46543,3))

In [ ]:
x.shape

In [ ]:
plt.imshow(x[46542], cmap = "gray")
plt.show()

In [ ]:
np.save('images_task3_%d_aug.npy'%IMAGE_SIZE,x)

In [ ]:
shape = images_MEL_aug.shape
new_images = np.empty((, shape[1], shape[2], shape[3]))

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
images = np.load('/home/wonjae/Classification_inception_v4/dataset/images_task3_299_aug.npy')
gt_labels = np.load('/home/wonjae/Classification_inception_v4/dataset/gt_task3_299_aug.npy')
data_size = images.shape[0]

In [ ]:
 # randomization.

randomized_indices = np.random.choice(data_size,data_size,replace=False)

In [ ]:
randomized_images=images[randomized_indices,:,:]

In [ ]:
randomized_labels = gt_labels[randomized_indices]

In [ ]:
randomized_labels.shape

In [ ]:
randomized_images.shape

In [ ]:
np.save('images_task3_%d_aug_ran.npy',randomized_images)
np.save('gt_task3_299_aug_ran.npy',randomized_labels)